# Check and display
A final check for data integrity. 
All labels and images are loaded and compared in size. 
Some labels and images are also visualy inspected. 

In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
import pandas as pd
import torch
import nrrd
from pathlib import Path
from monai.transforms import LoadImage
from trainlib.utils import load_config
from trainlib.viewer import BasicViewer, ListViewer
from trainlib.trainer import SegmentationTrainer
from trainlib.preprocessing import resample_to_ras_and_spacing
import SimpleITK as sitk
from tqdm import tqdm

In [53]:
config = load_config("../configs/binary.yaml")
trainer = SegmentationTrainer(config=config)

Setuptools is replacing distutils.


2022-11-03 14:48:27,104 - importing patch `AdaptAffineMatrixd` from `/home/bressekk/Documents/bone-tumor/patch/transforms.py`.
2022-11-03 14:48:27,107 - importing patch `RaiseOnEmptyd` from `/home/bressekk/Documents/bone-tumor/patch/transforms.py`.
2022-11-03 14:48:27,110 - importing patch `AdaptAffineMatrixd` from `/home/bressekk/Documents/bone-tumor/patch/transforms.py`.
2022-11-03 14:48:27,111 - importing patch `RaiseOnEmptyd` from `/home/bressekk/Documents/bone-tumor/patch/transforms.py`.
2022-11-03 14:48:27,247 - No pushover credentials file submitted, will not try to push trainings progress to pushover device. If you want to receive status updated via pushover, provide the path to a yaml file, containing the `app_token`, `user_key` and `proxies` (optional) in the config at `pushover_credentials`


In [54]:
trainer.data_loader.show_batch()

### Train/Valid/Test Split

In [55]:
import numpy as np

data = pd.read_csv(config.data.train_csv)
state = np.random.RandomState()
state.seed(42)
test = data.sample(frac=0.1, random_state=state)
data = data.drop(test.index)
valid = data.sample(frac=0.1, random_state=state)
train = data.drop(valid.index)

fn = config.data.train_csv.replace(".csv", "")
test.to_csv(fn + "_test.csv", index=False)
valid.to_csv(fn + "_valid.csv", index=False)
train.to_csv(fn + "_train.csv", index=False)